In [1]:
import pandas as pd
import sys
import json
from pprint import pprint
from elasticsearch import Elasticsearch

In [2]:
emails =  pd.read_csv('email_events_202101071339.csv')
orders =  pd.read_csv('orders_202101071341.csv')
products =  pd.read_csv('products_202101071344.csv')

In [3]:
orders_products = pd.merge(orders.copy(), products.copy(), how='left',left_on='product_id', right_on='product_id')
emails_orders_products = pd.merge(emails.copy(), orders_products.copy(), how='left',left_on='customer', right_on='customer')

In [4]:
len(emails_orders_products)

2661134

In [5]:
orders_products.to_csv('orders_products.csv', index=False)
emails_orders_products.to_csv('emails_orders_products.csv', index=False)

## Convert CSV To json

In [6]:
import csv 
import json 
  
# Function to convert a CSV to JSON 
# Takes the file paths as arguments 
def make_json(csvFilePath, jsonFilePath, id_name): 
      
    # create a dictionary 
    data = {} 
      
    # Open a csv reader called DictReader 
    with open(csvFilePath, encoding='utf-8') as csvf: 
        csvReader = csv.DictReader(csvf) 
          
        # Convert each row into a dictionary  
        # and add it to data 
        for rows in csvReader: 
              
            # Assuming a column named 'No' to 
            # be the primary key 
            key = rows[id_name] 
            data[key] = rows 
  
    # Open a json writer, and use the json.dumps()  
    # function to dump data 
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonf.write(json.dumps(data, indent=4)) 


In [7]:
csvFilePath = r'orders_products.csv'
jsonFilePath = r'orders_products.json'
id_name = 'order_id'
# Call the make_json function 
make_json(csvFilePath, jsonFilePath, id_name)

In [8]:
csvFilePath = r'emails_orders_products.csv'
jsonFilePath = r'emails_orders_products.json'
id_name = 'event_id'
make_json(csvFilePath, jsonFilePath, id_name)

## Bulk data in elasticsearch

In [9]:
import csv
from elasticsearch import helpers, Elasticsearch

def csv_reader(file_name, index_name):
    es = Elasticsearch([{'host': '127.0.0.1', 'port': 9200}])
    with open(file_name, 'r') as outfile:
        reader = csv.DictReader(outfile)
        helpers.bulk(es, reader, index=index_name, doc_type="Default")

In [10]:
csv_reader('emails_orders_products.csv', 'emails_orders_products')

C:\ProgramData\Anaconda3\lib\site-packages\elasticsearch\connection\base.py:193: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [11]:
csv_reader('orders_products.csv', 'orders_products')